In [ ]:
# %%
#import libraries
!pip install kaggle
import kaggle

# %%
!kaggle datasets download -d shudhanshusingh/az-medicine-dataset-of-india

# %%
#extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('az-medicine-dataset-of-india.zip') 
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file


# %%
import pandas as pd
df=pd.read_csv('az-medicine-dataset-of-india.zip')

# %%
df

# %%
# Define the lambda function to map substrings to values
def map_medicine_type(medicine):
    if 'tablets' in medicine:
        return 'Tablets'
    elif 'Suspension' in medicine:
        return 'Suspension'
    elif 'Injection' in medicine:
        return 'Injection'
    elif 'Capsule' in medicine:
        return 'Capsule'
    elif 'Syrup' in medicine:
        return 'Syrup'
    elif 'Oral' in medicine:
        return 'Oral'
    elif 'Ear' in medicine:
        return 'Ear Drop'
    elif 'Nasal' in medicine:
        return 'Nasal'
    elif 'gel' in medicine:
        return 'gel'
    elif 'Cream' in medicine:
        return 'Cream'
    else:
        return 'other'

# Apply the lambda function to the 'Medicine' column
df['Category'] = df['pack_size_label'].apply(lambda x: map_medicine_type(x))

df['short_composition1'].fillna('')



# %%
df['process_type'] = df['short_composition2'].apply(lambda x: 'Filling' if pd.notna(x) else 'Manufacturing')

df

# %%


# Apply the lambda function to the 'Medicine' column for getting strength
df['Strength']=df['short_composition1'].str.extract(r'\((.*?)\)')

# %%
# rename column for better understanding
df = df.rename(columns={'name': 'Product_Name','price(₹)':'Price','manufacturer_name':'Company','pack_size_label':'Size','Is_discontinued':'Active','short_composition1':'Composition1','short_composition2':'Composition2'})
df['Active']=df['Active'].apply(lambda x:1 if x==False else 0)
df = df.fillna('')


# %%
#connect to ssms
import sqlalchemy as sal
engine = sal.create_engine('mssql://owes\SQLEXPRESS/Pharma_Project?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

# %%
#load the data into sql server using append option
df.to_sql('medicine', con=conn , index=False, if_exists = 'append')

# %%
df

